# Import

In [31]:
import os
import cv2
import pandas as pd
import numpy as np
import random
from typing import List, Union
from joblib import Parallel, delayed

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from sklearn.model_selection import KFold

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

import datetime
import pytz
import matplotlib.pyplot as plt

# Config

In [35]:
config = {
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'base_path': '../open20/', # change relative path of data
    'train_data': 'zeroto20.csv', # change train data csv name
    'test_data': 'test20.csv', # change test data csv name
    'seed': 42,
    'valid_size': 0.1,
    'early_stopping': 30,
    'scheduler': True,
    'nums_pixel': False,
    'k-fold': 1, # 1이하의 정수이면 사용하지 않음
    'train' : {
       'batch_size' : 16,
       'num_workers': 1,
       'epochs': 300,
       'lr': 0.001,
    },
    'inference' : {
       'batch_size' : 8,
       'num_workers': 1,
       'threshold': 0.35,
    },
}

custom_transform = {
    'train':A.Compose([
        A.augmentations.crops.transforms.RandomCrop(224,224,p=1.0),
        A.RandomRotate90(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Normalize(),
        ToTensorV2()
    ]),
    'valid':A.Compose([
        A.augmentations.crops.transforms.CenterCrop(224,224,p=1.0),
        A.Normalize(),
        ToTensorV2()
    ]),
    'test': A.Compose([
        A.Normalize(),
        ToTensorV2()
    ]),
}

# Utils

In [33]:
# 시드 고정 함수
def fix_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

# RLE 디코딩 함수
def rle_decode(mask_rle: Union[str, int], shape=(224, 224)) -> np.array:
    '''
    mask_rle: run-length as string formatted (start length)
    shape: (height,width) of array to return
    Returns numpy array, 1 - mask, 0 - background
    '''
    if mask_rle == -1:
        return np.zeros(shape)

    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# dice score 계산 함수
def dice_score(prediction: np.array, ground_truth: np.array, smooth=1e-7) -> float:
    '''
    Calculate Dice Score between two binary masks.
    '''
    intersection = np.sum(prediction * ground_truth)
    return (2.0 * intersection + smooth) / (np.sum(prediction) + np.sum(ground_truth) + smooth)

def calculate_dice_scores(validation_df, img_shape=(224, 224)) -> List[float]:
    '''
    Calculate Dice scores for a dataset.
    '''
    # Extract the mask_rle columns
    pred_mask_rle = validation_df.iloc[:, 3]
    gt_mask_rle = validation_df.iloc[:, 4]

    def calculate_dice(pred_rle, gt_rle):
        pred_mask = rle_decode(pred_rle, img_shape)
        gt_mask = rle_decode(gt_rle, img_shape)
        if np.sum(gt_mask) > 0 or np.sum(pred_mask) > 0:
            return dice_score(pred_mask, gt_mask)
        else:
            return None  # No valid masks found, return None
    dice_scores = [calculate_dice(pred_rle, gt_rle) for pred_rle, gt_rle in zip(pred_mask_rle, gt_mask_rle)]
    dice_scores = [score for score in dice_scores if score is not None]  # Exclude None values
    return np.mean(dice_scores)

def calculate_nums_pixel(validation_df, img_shape=(224, 224)):
    '''
    Validation의 건물 pixel 수와 Prediction의 건물 pixel 수를 계산합니다.
    더 많이 예측하는지, 덜 예측하는지 기준을 잡고 threshold를 조정에 도움이 될 수 있습니다.
    '''
    eps = 1e-6
    batch_temp, count = 0, 0
    more_pred,less_pred = 0, 0
    pred_mask = validation_df.iloc[:, 2]
    gt_mask = validation_df.iloc[:, 3]
    for p_mask, t_mask in zip(pred_mask, gt_mask):
        if np.sum(rle_decode(t_mask, img_shape)):
            count += 1
            temp = float(int(np.sum(rle_decode(t_mask, img_shape)) - int(np.sum(rle_decode(p_mask, img_shape)))) / (int(np.sum(rle_decode(t_mask, img_shape))) + eps))
            if temp > 0: more_pred+= 1
            elif temp < 0: less_pred+= 1
            batch_temp += temp
    return batch_temp/count, more_pred, less_pred



class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        return 1 - dice

# Train
- 1024 * 1024 * 3 이미지 학습

### load train data

In [ ]:
# Colab 사용시 활성화
# 드라이브 마운트

# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Colab 사용시 활성화

# colab_base = '/content/drive/MyDrive/open'# 구글드라이브에서 데이터를 저장한 경로로 바꾸세요.
# train_df = pd.read_csv(f"{colab_base}/{config['train_data']}")
# train_df['img_path'] = colab_base + train_df['img_path'].str[1:]
# if config['k-fold'] > 1:
#     kfold = KFold(n_splits=config['k-fold'], random_state=config['seed'], shuffle=True)
#     print(kfold)
# else:
#     train, val = train_test_split(train_df, test_size=config['valid_size'], random_state=config['seed'])
#     print("train: ", len(train), "   valid: ", len(val))

In [ ]:
# kaggle 사용시 활성화

# kaggle_base = '/kaggle/input/swdacon' # kaggle에서 불러온 데이터의 경로로 바꾸세요.

# train_df = pd.read_csv(f"{kaggle_base}/{config['train_data']}")
# train_df = train_df[:20]
# train_df['img_path'] = kaggle_base + train_df['img_path'].str[1:]
# if config['k-fold'] > 1:
#     kfold = KFold(n_splits=config['k-fold'], random_state=config['seed'], shuffle=True)
#     print(kfold)
# else:
#     train, val = train_test_split(train_df, test_size=config['valid_size'], random_state=config['seed'])
#     print("train: ", len(train), "   valid: ", len(val))

In [36]:
# 로컬 사용시 활성화

train_df = pd.read_csv(f"{config['base_path']}/{config['train_data']}")
train_df['img_path'] = config['base_path'] + train_df['img_path'].str[1:]
if config['k-fold'] > 1:
    kfold = KFold(n_splits=config['k-fold'], random_state=config['seed'], shuffle=True)
    print(kfold)
else:
    train, val = train_test_split(train_df, test_size=config['valid_size'], random_state=config['seed'])
    print("train: ", len(train), "   valid: ", len(val))

train:  18    valid:  2


### Custom Dataset

In [37]:
class CustomDataset(Dataset):
    def __init__(self, img_paths, mask_rles = None, transform=None, infer=False):
        self.img_paths = img_paths
        self.mask_rles = mask_rles
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths.iloc[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image

        mask_rle = self.mask_rles.iloc[idx]
        mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

### Data Loader

In [38]:
fix_seed(config['seed'])

if config['k-fold'] > 1:
    print(f"k-fold: {config['k-fold']} 입력되어, training할 때 fold별 dataloader가 생성됩니다.")
else:
    train_dataset = CustomDataset(img_paths=train['img_path'], mask_rles=train['mask_rle'], transform=custom_transform['train'])
    train_dataloader = DataLoader(train_dataset, batch_size=config['train']['batch_size'], shuffle=True)

    valid_dataset = CustomDataset(img_paths=val['img_path'], mask_rles=val['mask_rle'], transform=custom_transform['valid'])
    valid_dataloader = DataLoader(valid_dataset , batch_size=config['train']['batch_size'], shuffle=False)

### Model
- 사용하는 모델에 따라 바꾸어야 합니다. 필요한 경우 라이브러리를 install 해야 합니다.
- baseline ver2.1의 모델은 현재 최고점인 TransUNet입니다.

In [ ]:
# !pip install vit_pytorch

In [ ]:
from vit_pytorch import ViT
import torchvision.models as models
from einops.layers.torch import Rearrange, Reduce

In [ ]:
vit_encoder = ViT(
    image_size=14,
    patch_size=1,
    dim = 128,
    depth=12,
    num_classes=2,
    heads=8,
    mlp_dim=2048,
)
vit_encoder=vit_encoder.to(config['device'])
# depth=12, head=8일때가 현재 최고설정

In [ ]:
def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True)
    )
class TransUnet_b5(nn.Module):
    def __init__(self):
        super(TransUnet_b5, self).__init__()
        self.backbone = models.efficientnet_b5(weights='EfficientNet_B5_Weights.DEFAULT')
        self.vit_flatten = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=1, p2=1),
            nn.LayerNorm(128)
        )
        self.conv_vit_res = nn.Sequential(
            nn.Conv2d(128,64,3,1,1),
            nn.ReLU(inplace=True)
        )
        self.res_conv = nn.Sequential(
            nn.Conv2d(16,16,3,1,1),
            nn.ReLU(inplace=True)
        )
        self.seg_conv = nn.Conv2d(16, 1, 1)
        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.dconv_up3 = double_conv(64 + 64, 64)
        self.dconv_up2 = double_conv(40 + 64, 32)
        self.dconv_up1 = double_conv(32 + 24, 16)

        self.conv_last = nn.Conv2d(16, 1, 1)
    def forward(self, x):
        x = self.backbone.features[0](x) #48,112,112
        out1 = self.backbone.features[1](x) #24,112,112
        out2 = self.backbone.features[2](out1) #40,56,56
        out3 = self.backbone.features[3](out2) #64,28,28
        out4 = self.backbone.features[4](out3) #128,14,14

        vit_out = self.vit_flatten(out4)
        vit_out = vit_encoder.transformer(vit_out)
        vit_out = vit_out.reshape(-1,128,14,14)
        vit_out = self.conv_vit_res(vit_out) #64,14,14
        up3 = self.upsample(vit_out)#64,28,28
        up3 = torch.cat([up3, out3], dim=1) #128,28,28
        up3 = self.dconv_up3(up3) #64,28,28

        up2 = self.upsample(up3) #64,56,56
        up2 = torch.cat([up2, out2], dim=1) #40+64,56,56
        up2 = self.dconv_up2(up2) #32,56,56

        up1 = self.upsample(up2) #32,112,112
        up1 = torch.cat([up1,out1],dim=1) #32+24,112,112
        up1 = self.dconv_up1(up1) #16,112,112

        up0 = self.upsample(up1)#16,224,224
        up0 = self.res_conv(up0)
        res = self.seg_conv(up0)
        return res

In [ ]:
if config['k-fold'] > 1:
    print(f"k-fold: {config['k-fold']} 입력되어, training할 때 fold별 model이 생성됩니다.")
else:
    model = TransUnet_b5().to(config['device'])

In [39]:
# val 사진 띄우기 셀 만들기용 가벼운 모델 
import segmentation_models_pytorch as smp

model = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)\
)


### Validation

In [40]:
def validation(config, model, criterion, valid_loader, val):
    model.eval()
    valid_loss = 0
    result = []
    transformed_mask = []
    val_df = val.copy()

    with torch.no_grad():
        for images, masks in tqdm(valid_loader):
            if type(transformed_mask) == torch.Tensor:
                transformed_mask = torch.cat([transformed_mask, masks])
            else:
                transformed_mask = masks.clone().detach()
            images = images.float().to(config['device'])
            masks = masks.float().to(config['device'])

            outputs = model(images)
            loss = criterion(outputs, masks.unsqueeze(1))
            valid_loss += loss.item()

            output_masks = torch.sigmoid(outputs).cpu().numpy()
            output_masks = np.squeeze(output_masks, axis=1)
            output_masks = (output_masks > config['inference']['threshold']).astype(np.uint8)

            for i in range(len(images)):
                mask_rle = rle_encode(output_masks[i])
                if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                    result.append(-1)
                else:
                    result.append(mask_rle)
        val_df['valid_mask_rle'] = result
        val_df['transformed_mask_rle'] = list(map(rle_encode, transformed_mask.squeeze().numpy()))
        dice_score = calculate_dice_scores(val_df)
        if config['nums_pixel']:
            mean_error_ratio, more_pred, less_pred = calculate_nums_pixel(val_df)
    if config['nums_pixel']:
        return valid_loss/len(valid_loader), dice_score, mean_error_ratio, more_pred, less_pred
    else:
        return valid_loss/len(valid_loader), dice_score

### Train

In [41]:
def training(config, model, train_loader, valid_loader, val, fold=0):
    model = model.to(config['device'])
    es_count = 0
    min_val_loss = float('inf')
    best_model = None

    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config['train']['lr'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, min_lr=1e-8, verbose=True)
    print("***TRAINING START***")
    # training loop
    for epoch in range(config['train']['epochs']):
        model.train()
        epoch_loss = 0
        for images, masks in tqdm(train_loader):
            images = images.float().to(config['device'])
            masks = masks.float().to(config['device'])

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks.unsqueeze(1))
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        if config['nums_pixel']:
            val_loss, dice_score, mean_error_ratio, more_pred, less_pred  = validation(config, model, criterion, valid_loader, val)
        else:
            val_loss, dice_score = validation(config, model, criterion, valid_loader, val)

        es_count += 1
        if min_val_loss > val_loss:
            es_count = 0
            min_val_loss = val_loss
            best_model = model
            state_dict = model.state_dict()
            best_epoch = epoch + 1
            print(f"Epoch [{epoch + 1}] New Minimum Valid Loss!")
            if epoch+1 > 50: # 몇 epoch 이상부터 저장할지 결정합니다.
                if fold:
                    current_model = f"fold{fold}_epoch{epoch+1}_current_model.pt"
                else:
                    current_model = f"epoch{epoch+1}_current_model.pt"
                torch.save(state_dict, f'{config["base_path"]}/{current_model}') # 비정상적인 종료에 대비해 최고점일 때 모델을 저장하고 싶으면 경로를 지정하고 활성화하십시오.

        if config['scheduler']:
            scheduler.step(val_loss)

        if es_count == config['early_stopping']:
            if config['nums_pixel']:
                print(f'Epoch {epoch+1}, Train Loss: {(epoch_loss/len(train_loader)):6f}, Valid Loss: {val_loss:6f}, ES Count: {es_count}')
                print(f'Dice Coefficient: {dice_score:6f}, (GT - Pred)/GT: {mean_error_ratio:2f}, More Pred : Less Pred = {more_pred} : {less_pred}')
            else:
                print(f'Epoch {epoch+1}, Train Loss: {(epoch_loss/len(train_loader)):6f}, Valid Loss: {val_loss:6f}, Dice Coefficient: {dice_score:6f}, ES Count: {es_count}')
            print(f"Early Stopping Count에 도달하지 않았습니다! \nEarly Stopping Count: {config['early_stopping']} Best Epoch: {best_epoch}")
            print("***TRAINING DONE***")
            return best_model, state_dict

        if config['nums_pixel']:
            print(f'Epoch {epoch+1}, Train Loss: {(epoch_loss/len(train_loader)):6f}, Valid Loss: {val_loss:6f}, ES Count: {es_count}')
            print(f'Dice Coefficient: {dice_score:6f}, (GT - Pred)/GT: {mean_error_ratio:2f}, More Pred : Less Pred = {more_pred} : {less_pred}')
        else:
            print(f'Epoch {epoch+1}, Train Loss: {(epoch_loss/len(train_loader)):6f}, Valid Loss: {val_loss:6f}, Dice Coefficient: {dice_score:6f}, ES Count: {es_count}')
        print("------------------------------------------------------------------------------------")

    print(f"Early Stopping Count에 도달하지 않았습니다! \nEarly Stopping Count: {config['early_stopping']} Best Epoch: {best_epoch}")
    print("***TRAINING DONE***")
    return best_model, state_dict

In [42]:
torch.cuda.is_available() # 학습 전에 GPU 쓰고 있나 확인

True

In [43]:
fix_seed(config['seed'])

if config['k-fold'] > 1:
    best_models, best_model_state_dicts = [], []
    for i, (train_idx, val_idx) in enumerate(kfold.split(train_df)):
        train = train_df.iloc[train_idx,:]
        val = train_df.iloc[val_idx,:]
        train_dataset = CustomDataset(img_paths=train['img_path'], mask_rles=train['mask_rle'], transform=custom_transform['train'])
        train_dataloader = DataLoader(train_dataset, batch_size=config['train']['batch_size'], shuffle=True)

        valid_dataset = CustomDataset(img_paths=val['img_path'], mask_rles=val['mask_rle'], transform=custom_transform['valid'])
        valid_dataloader = DataLoader(valid_dataset , batch_size=config['train']['batch_size'], shuffle=False)
        print(f"--- Start Fold {i + 1} ---")
        model = TransUnet_b5().to(config['device'])
        best_model, best_model_state_dict = training(config, model, train_dataloader, valid_dataloader, val, i+1)
        best_models.append(best_model)
        best_model_state_dicts.append(best_model)
        print(f"---- End Fold {i + 1} ----")
else:
    best_model, best_model_state_dict = training(config, model, train_dataloader, valid_dataloader, val)

***TRAINING START***


100%|██████████| 1/1 [00:00<00:00, 14.18it/s]


Epoch [1] New Minimum Valid Loss!
Epoch 1, Train Loss: 0.959589, Valid Loss: 25.165754, Dice Coefficient: 0.005342, ES Count: 0
------------------------------------------------------------------------------------


100%|██████████| 1/1 [00:00<00:00, 16.05it/s]


Epoch 2, Train Loss: 0.793203, Valid Loss: 25.733156, Dice Coefficient: 0.005338, ES Count: 1
------------------------------------------------------------------------------------


...

Epoch 114, Train Loss: 0.129931, Valid Loss: 0.040041, Dice Coefficient: 0.405239, ES Count: 28
------------------------------------------------------------------------------------


100%|██████████| 1/1 [00:00<00:00, 19.00it/s]


Epoch 115, Train Loss: 0.193660, Valid Loss: 0.039079, Dice Coefficient: 0.419277, ES Count: 29
------------------------------------------------------------------------------------


100%|██████████| 1/1 [00:00<00:00, 16.37it/s]

Epoch 00116: reducing learning rate of group 0 to 7.8125e-06.
Epoch 116, Train Loss: 0.114372, Valid Loss: 0.039732, Dice Coefficient: 0.418269, ES Count: 30
Early Stopping Count에 도달하지 않았습니다! 
Early Stopping Count: 30 Best Epoch: 86
***TRAINING DONE***


## Inference
- 224 * 224 * 3 이미지 추론

### load test data

In [ ]:
# Colab 사용시 활성화

# test_df = pd.read_csv(f"{colab_base}/{config['test_data']}")
# test_df['img_path'] = colab_base + test_df['img_path'].str[1:]

In [ ]:
# kaggle 사용시 활성화

# test_df = pd.read_csv(f"{kaggle_base}/{config['test_data']}")
# test_df['img_path'] = kaggle_base + test_df['img_path'].str[1:]

In [44]:
# 로컬 사용시 활성화

test_df = pd.read_csv(f"{config['base_path']}/{config['test_data']}")
test_df['img_path'] = config['base_path'] + test_df['img_path'].str[1:]

### Data Loader

In [45]:
fix_seed(config['seed'])

if config['k-fold'] > 1:
    print(f"k-fold: {config['k-fold']} 입력되어, inference할 때 fold별 dataloader가 생성됩니다.")
else:
    test_dataset = CustomDataset(img_paths=test_df['img_path'], transform=custom_transform['test'], infer=True)
    test_dataloader = DataLoader(test_dataset, batch_size=config['inference']['batch_size'], shuffle=False)

In [51]:
def inference(config, model, test_loader):
    with torch.no_grad():
        model.eval()
        result = []
        for images in tqdm(test_loader):
            images = images.float().to(config['device'])

            outputs = model(images)
            masks = torch.sigmoid(outputs).cpu().numpy()
            masks = np.squeeze(masks, axis=1)
            masks = (masks > config['inference']['threshold']).astype(np.uint8)

            for i in range(len(images)):
                mask_rle = rle_encode(masks[i])
                if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                    result.append(-1)
                else:
                    result.append(mask_rle)
    return result

In [47]:
if config['k-fold'] > 1:
    inference_results = []
    for i, fold_model in enumerate(best_models):
        test_dataset = CustomDataset(img_paths=test_df['img_path'], transform=custom_transform['test'], infer=True)
        test_dataloader = DataLoader(test_dataset, batch_size=config['inference']['batch_size'], shuffle=False)
        print(f"--- Start Fold {i + 1} ---")
        inference_result = inference(config, fold_model, test_dataloader)
        inference_results.append(inference_result)
else:
    inference_result = inference(config, best_model, test_dataloader)

100%|██████████| 3/3 [00:00<00:00,  3.62it/s]


## valid 성능확인을 위한 셀

In [52]:
pred_dataset = CustomDataset(img_paths=train_df['img_path'], transform=custom_transform['valid'], infer=True)
pred_dataloader = DataLoader(pred_dataset, batch_size=config['inference']['batch_size'], shuffle=False)
inference_result = inference(config, best_model, pred_dataloader)

kst = pytz.timezone('Asia/Seoul')
now = datetime.datetime.now(tz=kst)
current_time = now.strftime("%y%m%d-%H_%M_%S")

submit = pd.read_csv(f"{config['base_path']}/submit20.csv")
submit['mask_rle'] = inference_result
file_name = f"valid_{current_time}.csv"
submit.to_csv(f"{config['base_path']}/{file_name}", index=False)

100%|██████████| 3/3 [00:00<00:00,  5.45it/s]


In [64]:
def pred_viewer(train_df, valid_csv, img_num):
    """
    white -> 건물 black -> 배경
    1. Local에서 사용 시 train_df, pred_csv, img_num만 입력
    2. colab에서 사용 시 아래 사항을 입력
    base_path = colab_base
    is_colab = True
    """
    mask_rle = train_df.iloc[img_num, 2]
    image_path = train_df.iloc[img_num, 1]
    pred = valid_csv.iloc[img_num, 1]
    image = cv2.imread(image_path)
    mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))
    pred = rle_decode(pred, (image.shape[0], image.shape[1]))
    fig = plt.figure()
    ax1 = fig.add_subplot(1,3,1)
    ax1.imshow(image)
    ax1.set_title('image')
    ax2 = fig.add_subplot(1,3,3)
    ax2.imshow(mask,cmap='gray')
    ax2.set_title('mask')
    plt.show()

In [63]:
valid_csv

,img_id,mask_rle
0,TEST_00000,b''
1,TEST_00001,b'1 1 1186 5 1407 15 1630 18 1853 20 2075 22 2...
2,TEST_00002,b'4454 5 4672 18 4894 23 5117 26 5341 28 5564 ...
3,TEST_00003,b'62 2 65 2 951 1 959 6 1173 18 1396 21 1620 2...
4,TEST_00004,b'204 16 435 1 656 6 878 12 1101 16 1325 17 15...
5,TEST_00005,b'59 31 186 32 285 15 412 25 507 23 634 28 730...
6,TEST_00006,b''
7,TEST_00007,b'1 1 1802 2 2025 4 2248 6 2472 6 2696 6 2920 ...
8,TEST_00008,b''
9,TEST_00009,b'1 1 6 13 159 25 460 1 610 12 679 11 833 18 9...


In [65]:
valid_csv = pd.read_csv(f"{config['base_path']}/{file_name}")

pred_viewer(train_df, valid_csv, 0)

ValueError: invalid literal for int() with base 10: "b''"

## Submission

In [ ]:
# Colab 사용시 활성화

# kst = pytz.timezone('Asia/Seoul')
# now = datetime.datetime.now(tz=kst)
# current_time = now.strftime("%y%m%d-%H_%M_%S")

# if config['k-fold'] > 1:
#     for i,inference_result in enumerate(inference_results):
#         submit = pd.read_csv(f"{colab_base}/sample_submission.csv")
#         submit['mask_rle'] = inference_result
#         file_name = f"fold_{i+1}_{current_time}.csv"
#         submit.to_csv(f"{colab_base}/{file_name}", index=False)

#     # 모델 저장
#     for i,best_model_state_dict in enumerate(best_model_state_dicts):
#         model_name = f"fold_{i+1}_{current_time}.pt"
#         torch.save(best_model, f'{colab_base}/{model_name}')
# else:
#     submit = pd.read_csv(f"{colab_base}/sample_submission.csv")
#     submit['mask_rle'] = inference_result
#     file_name = f"{current_time}.csv"
#     submit.to_csv(f"{colab_base}/{file_name}", index=False)

#     # 모델 저장
#     model_name = f"{current_time}.pt"
#     torch.save(best_model, f'{colab_base}/{model_name}')
# print(f"{file_name}")

In [ ]:
# kaggle 사용시 활성화

# kst = pytz.timezone('Asia/Seoul')
# now = datetime.datetime.now(tz=kst)
# current_time = now.strftime("%y%m%d-%H_%M_%S")

# if config['k-fold'] > 1:
#     for i,inference_result in enumerate(inference_results):
#         submit = pd.read_csv(f"{kaggle_base}/sample_submission.csv")
#         submit = submit[:20]
#         submit['mask_rle'] = inference_result
#         file_name = f"fold_{i+1}_{current_time}.csv"
#         submit.to_csv(f"/kaggle/working/{file_name}", index=False)

#     # 모델 저장
#     for i,best_model_state_dict in enumerate(best_model_state_dicts):
#         model_name = f"fold_{i+1}_{current_time}.pt"
#         torch.save(best_model, f'/kaggle/working/{model_name}')
# else:
#     submit = pd.read_csv(f"{kaggle_base}/sample_submission.csv")
#     submit['mask_rle'] = inference_result
#     file_name = f"{current_time}.csv"
#     submit.to_csv(f"/kaggle/working/{file_name}", index=False)

#     # 모델 저장
#     model_name = f"{current_time}.pt"
#     torch.save(best_model, f'/kaggle/working/{model_name}')
# print(f"{file_name}")

In [ ]:
# 로컬 사용시 활성화

kst = pytz.timezone('Asia/Seoul')
now = datetime.datetime.now(tz=kst)
current_time = now.strftime("%y%m%d-%H_%M_%S")

if config['k-fold'] > 1:
    for i,inference_result in enumerate(inference_results):
        submit = pd.read_csv(f"{config['base_path']}/sample_submission.csv")
        submit['mask_rle'] = inference_result
        file_name = f"fold_{i+1}_{current_time}.csv"
        submit.to_csv(f"{config['base_path']}/{file_name}", index=False)

    # 모델 저장
    for i,best_model_state_dict in enumerate(best_model_state_dicts):
        model_name = f"fold_{i+1}_{current_time}.pt"
        torch.save(best_model, f"{config['base_path']}/{model_name}")
else:
    submit = pd.read_csv(f"{config['base_path']}/sample_submission.csv")
    submit['mask_rle'] = inference_result
    file_name = f"{current_time}.csv"
    submit.to_csv(f"{config['base_path']}/{file_name}", index=False)

    # 모델 저장
    model_name = f"{current_time}.pt"
    torch.save(best_model, f"{config['base_path']}/{model_name}")
print(f"{file_name}")

# Submission Viewer
- fold 사용시엔 사용할 필요 x

In [68]:
def submission_viewer(test_csv, submit_csv, img_num):
    """
    white -> 건물 black -> 배경
    1. Local에서 사용 시 test_csv, submit_csv, img_num만 입력
    2. colab에서 사용 시 아래 사항을 입력
    base_path = colab_base
    is_colab = True
    """
    mask_rle = submit_csv.iloc[img_num, 1]
    image_path = test_csv.iloc[img_num, 1]
    image = cv2.imread(image_path)
    mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))
    fig = plt.figure()
    ax1 = fig.add_subplot(1,2,1)
    ax1.imshow(image)
    ax1.set_title('image')
    ax2 = fig.add_subplot(1,2,2)
    ax2.imshow(mask,cmap='gray')
    ax2.set_title('mask')
    plt.show()

In [ ]:
# Colab 사용시 활성화

# if config['k-fold'] > 1:
#     print("Ensemble을 진행하십시오.")
# else:
#     last_submit = pd.read_csv(f"{colab_base}/{file_name}")
#     submission_viewer(test_df, last_submit, 0)
#     submission_viewer(test_df, last_submit, 1)
#     submission_viewer(test_df, last_submit, 2)

In [ ]:
# kaggle 사용시 활성화

# if config['k-fold'] > 1:
#     print("Ensemble을 진행하십시오.")
# else:
#     last_submit = pd.read_csv(f"/kaggle/working/{file_name}")
#     submission_viewer(test_df, last_submit, 0)
#     submission_viewer(test_df, last_submit, 1)
#     submission_viewer(test_df, last_submit, 2)

In [66]:
last_submit

,img_id,mask_rle
0,TEST_00000,b''
1,TEST_00001,b'1 1 1186 5 1407 15 1630 18 1853 20 2075 22 2...
2,TEST_00002,b'4454 5 4672 18 4894 23 5117 26 5341 28 5564 ...
3,TEST_00003,b'62 2 65 2 951 1 959 6 1173 18 1396 21 1620 2...
4,TEST_00004,b'204 16 435 1 656 6 878 12 1101 16 1325 17 15...
5,TEST_00005,b'59 31 186 32 285 15 412 25 507 23 634 28 730...
6,TEST_00006,b''
7,TEST_00007,b'1 1 1802 2 2025 4 2248 6 2472 6 2696 6 2920 ...
8,TEST_00008,b''
9,TEST_00009,b'1 1 6 13 159 25 460 1 610 12 679 11 833 18 9...


In [70]:
# 로컬 사용시 활성화

if config['k-fold'] > 1:
    print("Ensemble을 진행하십시오.")
else:
    last_submit = pd.read_csv(f"{config['base_path']}/{file_name}")
    submission_viewer(test_df, last_submit, 0)
    submission_viewer(test_df, last_submit, 1)
    submission_viewer(test_df, last_submit, 2)

ValueError: invalid literal for int() with base 10: "b''"